In [ ]:
#Import Dependencies
import requests
from bs4 import BeautifulSoup
import csv

In [ ]:
#The purpose of this code is to pull a list of possible dates and regions for weekly spotify charts us in 2019.
region_codes=[]
region_names=[]

date_codes=[]
date_names=[]

Original_url="https://spotifycharts.com/regional/global/weekly/latest"
list_response=requests.get(Original_url)
soup=BeautifulSoup(list_response.text, "lxml")
#I can add to this later to change from daily or choose another date.
dropdowns=soup.find_all('div', class_='responsive-select')

regions=dropdowns[0].find_all('li')
for region in regions:
    region_codes.append(region['data-value'])
    region_names.append(region.text)

dates=dropdowns[2].find_all('li')
for date in dates:
    date_codes.append(date['data-value'])
    date_names.append(date.text)

In [ ]:
#This is to limit the list of dates to 2019
for i in range(len(date_codes)-1,-1,-1):
    if date_codes[i][:4]!="2019":
        date_codes.pop(i)
        date_names.pop(i)

In [ ]:
#Open CSV
outPath="Resources/2019-weekly-regional-charts.csv"
with open(outPath, 'w', newline='') as csvfile:
    # Initialize csv.writer
    csvwriter = csv.writer(csvfile, delimiter=',')
    #Creating a header for the data
    csvwriter.writerow(["Region","Date","Position","Track Name","Artist","Streams","ID"])

#Looping through the region codes and using them in the url to scrape the csv
    for Num,Reg in enumerate(region_codes):
        Current_Region=region_names[Num]
        #Looping through Date codes
        for Num2,date in enumerate(date_codes):
            Current_Date=date_names[Num2]
            url=f"https://spotifycharts.com/regional/{Reg}/daily/{date}/download"
            CSVresponse=requests.get(url)
            Cleaned_Response=str(CSVresponse.content).split(chr(92)+"n")
            Cleaned_Response.pop(len(Cleaned_Response)-1)
            Cleaned_Response.pop(1)       
            Cleaned_Response.pop(0)
            reader= csv.reader(Cleaned_Response,delimiter=",")
            for row in reader:
                if row[0]=="    <head>":
                    break
                CleanRow=[Current_Region, Current_Date] + row
                CleanRow[6]=CleanRow[6][31:]
                csvwriter.writerow(CleanRow)